In [4]:
import sources.logins as lg
from time import sleep
import warnings
import pandas as pd
import os
import xlwings as xw
from datetime import datetime as dt
warnings.filterwarnings('ignore')
import pathlib

In [2]:

def login_exfm(driver_on = True):
    # generate key using fernet
    # key = Fernet.generate_key().decode()
    key = 'cLAiAdZU1U0sxWWK8sxhF0IWIBP4KrR3xhdu3x1EDQI='
    
    
    # login ExFM and download latest incompleted data
    name,pw = lg.save_id('exfm.txt',key)
    driver = lg.login_url('exfm',name,pw)
    
    
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search	
            break
        except Exception as e:
            print ('Wrong Username or Password. Try again...')
            name,pw = lg.save_id('exfm.txt',key)
            driver = lg.login_url('exfm',name,pw)
    
    d_type_menu = [
                    'Incompleted',
                    'History',
                    'Equipments',
                    'Customers',
                    'No Download'
                    
                    ]
    #border table
    print(f'\n{"_"*50}')
    print(f'{"|  No.|  Function": <49}|')
    print(f'|{"_"*48}|')
    for i in range(1,1+len(d_type_menu)):
        print(f'|{i: >3}  |  {d_type_menu[i-1]: <40}|')
    print(f'|{"_"*48}|') #bottom border
    while True:
        ind = str(input('Select Dataset to download: '))
        try:
            ind = int(ind)
        except:
            if ind.upper() == 'Q' or ind.upper() == 'QUIT':
                break
            else:
                print('Only accept number')
                continue
        if 0<ind<=len(d_type_menu):
            d_type = d_type_menu[ind-1]
            break
        else:
            print(f'Input number must be less than {len(d_type_menu)}\n')
    
    print(d_type)
    
    # driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search
    
    if d_type == d_type_menu[0]: # Incompleted
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[1]: # History
        Select(driver.find_element_by_xpath('//*[@id="sidIN_REPAIR_STATUS"]')).select_by_index(0)
        print('Select Status "All" and download.')
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[2]: # Equipments
        driver.find_element_by_xpath('//*[@id="EQP_MGT_LINK"]').click() # Equipments
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    
    if d_type == d_type_menu[3]: # Customers
        driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click() 
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    
    else:
        print('No Download')

    # check latest file
    i_wait = 0
    while True:
        try: # check file already
            file, ctime = lg.file_latest(folder_name='Downloads')
            if file.endswith('xls'):
                print (f'\nDonwload file {file} succesful at {ctime}')
                break
            else:
                print(f'Please wait for compressing file {file}...{i_wait}s')
                i_wait +=3
                sleep(3)
        except Exception as e: #waiting for download
            print(f'Please wait for dowloading...{i_wait}s')
            i_wait +=3
            sleep(3)
            if d_type == d_type_menu[-1]: break
    if driver_on:
        return driver
    else:
        #finish close driver
        driver.close()
    sleep(2)

def auto_accept(driver):
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    driver.switch_to.alert.accept()
    
def export_all_translog_in_page(driver):
    for i in range(50):
        # translog button
        driver.find_element_by_xpath('//*[@id="sidTRANSLOG_BUTTON_IMAGE"]').click()
        p = driver.current_window_handle
        chwd = driver.window_handles
        for w in chwd:
            if(w!=p):
                driver.switch_to.window(w)
                break
        sn = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table/tbody/tr[1]/td[1]').get_attribute('innerHTML')
        rma = driver.find_element_by_xpath('/html/body/div[1]/div[2]/table/tbody/tr[1]/td[2]').get_attribute('innerHTML')
        print(i+1,sn,rma)
        # click export
        driver.find_element_by_xpath('//*[@id="export"]').click()
        driver.close()
        sleep(0.5)
        driver.switch_to.window(p)

        # next button
        try:
            next_button = driver.find_element_by_xpath('//*[@id="sidSHOW_NEXT_REPAIR"]')
            next_button.click()
        except Exception as e:
            print(f"Finish all {i+1} jobs in this page.\n")
            break

In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import math

In [ ]:

driver = login_exfm()

In [ ]:
#RMA search tab
driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click()

In [ ]:
#show all
driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()
sleep(0.5)
ttl = driver.find_element_by_xpath('//*[@id="TH_search_result"]/thead/tr[1]/th/div/span').get_attribute('innerHTML')
ttl = int(ttl.split('/')[1][:-1])
print(ttl)

In [ ]:
# driver.switch_to.window(p)

In [ ]:
pg = math.ceil(ttl/50)
for i in range(pg):
    first_row = i * 50
    #Fisrt row
    driver.find_element_by_xpath(f'//*[@id="search_result[{first_row}].sidRS_REPAIR_ID_data"]').click()
    # export translog
    print(f'Export for page {i + 1}/{pg}')
    export_all_translog_in_page(driver)
    # return
    driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
    sleep(0.5)
    # next page
    driver.find_element_by_xpath('//*[@id="_eventcursornextpage_search_result"]').click()
    sleep(0.5)

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

### Read excel file

In [40]:
standard_format = {'RMA':'','SN':'',
              'Receive':'','Receiver':'',
              'Inspection':'','Inspector':'',
              'Report':'','Reporter':'',
              'Verify':'','Verifier':'',
              'Quotation':'','Admin':'',
              'Confirmation':'','FSS':'',
              'Check_Stock':'','Stock_Keeper':'',
              'PO_Create':'','Admin2':'',
              'APAS repair':'','Admin3':'',
              'APAS approved':'','Admin4':'',
                
              }
def read_translog(df):
    job_translog = {'RMA':'','SN':'',
              'Receive':'','Receiver':'',
              'Inspection':'','Inspector':'',
              'Report':'','Reporter':'',
              'Verify':'','Verifier':'',
              'Quotation':'','Admin':'',
              'Confirmation':'','FSS':'',
              'Check_Stock':'','Stock_Keeper':'',
              'PO_Create':'','Admin2':'',
              'APAS repair':'','Admin3':'',
              'APAS approved':'','Admin4':'',
                
              }
    # sn = df['S/N'][0]

    job_translog['RMA'] = df['RMA#'][0]
    job_translog['SN'] = df['S/N'][0]
    for i in range(len(df)):
        # 1. Receive
        if df['Screen'][i]=='Shipping Incoming' and df['Type'][i]=='Completion':
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Receive':dtime.strftime('%Y-%m-%d'),'Receiver':df['User'][i]})
    #         job_translog['Receive'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]

        # 2. Inspection
        if df['Screen'][i]=='Inspection' and df['Type'][i]=='Completion':
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Inspection':dtime.strftime('%Y-%m-%d'),'Inspector':df['User'][i]})

        # 3. Report Done
        if  df['Updated Item(After)'][i] in ('Status Info:"Waiting for Next Process Available"',''):
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Report':dtime.strftime('%Y-%m-%d'),'Reporter':df['User'][i]})

        # 4. Report Verified
        if  df['Updated Item(After)'][i] in ('Status Info:"Awating Parts billing"',''):
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Verify':dtime.strftime('%Y-%m-%d'),'Verifier':df['User'][i]})

        # 5. Quote Sent
        if  df['Updated Item(After)'][i] in ('Status Info:"Awating Contract billing"',''):
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
            job_translog.update({'Quotation':dtime.strftime('%Y-%m-%d'),'Admin':df['User'][i]})

        # 6. Confirmation
        if  'Parts Preparation (WH)' in df['Updated Item(After)'][i]:
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
    #         job_translog['Checked parts'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]
            job_translog.update({'Confirmation':dtime.strftime('%Y-%m-%d'),'FSS':df['User'][i]})

        # 7. Check stock
        if  'Waiting for Parts' in df['Updated Item(After)'][i] or 'Parts on Order' in df['Updated Item(After)'][i]:
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
    #         job_translog['Checked parts'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]
            job_translog.update({'Check_Stock':dtime.strftime('%Y-%m-%d'),'Stock_Keeper':df['User'][i]})

        # 8. PO Created
        if  'Parts on Order' in df['Updated Item(After)'][i]:
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
    #         job_translog['PO Created'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]
            job_translog.update({'PO_Create':dtime.strftime('%Y-%m-%d'),'Admin2':df['User'][i]})

        # 9. APAS repair
        if  df['Screen'][i]=='Parts Selection' and 'Awaiting Internal Approval' in df['Updated Item(After)'][i]:
            dtime = dt.strptime(df['Time'][i],'%Y/%m/%d %H:%M:%S')
    #         job_translog['APAS Repair'] = [dtime.strftime('%Y-%m-%d'),df['User'][i]]
            job_translog.update({'APAS Repair':dtime.strftime('%Y-%m-%d'),'Verifier':df['Admin3'][i]})

        # 10. APAS Approved
    #     if 


    return job_translog

In [38]:
display(standard_format)

{'RMA': '',
 'SN': '',
 'Receive': '',
 'Receiver': '',
 'Inspection': '',
 'Inspector': '',
 'Report': '',
 'Reporter': '',
 'Verify': '',
 'Verifier': '',
 'Quotation': '',
 'Admin': '',
 'Confirmation': '',
 'FSS': '',
 'Check_Stock': '',
 'Stock_Keeper': '',
 'PO_Create': '',
 'Admin2': '',
 'APAS repair': '',
 'Admin3': '',
 'APAS approved': '',
 'Admin4': ''}

In [ ]:
# read file to pandas

from datetime import datetime as dt
file = 'translog/export - 2023-10-12T215337.516.csv'
df = pd.read_csv(file,encoding = 'utf-8')
df

In [41]:
folder_name = 'translog'
files = os.listdir('translog')
rma_dict={}

full_translog =pd.DataFrame(columns=['RMA','SN',
                                      'Receive','Receiver',
                                      'Inspection','Inspector',
                                      'Report','Reporter',
                                      'Verify','Verifier',
                                      'Quotation','Admin',
                                      'Confirmation','FSS',
                                      'Check_Stock','Stock_Keeper',
                                      'PO_Create','Admin2',
                                      'APAS repair','Admin3',
                                      'APAS approved','Admin4',
                                      
                                        ])

for file in files:
#     file = 'Downloads/export (99).csv'
    df = pd.read_csv(f'{folder_name}/{file}',encoding = 'utf-8')
    try:
#         display(standard_format)
        trans_log = read_translog(df)
        rma = trans_log['RMA']
        rma_dict.update({rma:file})
    except Exception as e:
        print(e)
        
            
    full_translog = full_translog.append({
        'RMA':trans_log['RMA'],'SN':trans_log['SN'],
          'Receive':trans_log['Receive'],'Receiver':trans_log['Receiver'],
          'Inspection':trans_log['Inspection'],'Inspector':trans_log['Inspector'],
          'Report':trans_log['Report'],'Reporter':trans_log['Reporter'],
          'Verify':trans_log['Verify'],'Verifier':trans_log['Verifier'],
          'Quotation':trans_log['Quotation'],'Admin':trans_log['Admin'],
          'Confirmation':trans_log['Confirmation'],'FSS':trans_log['FSS'],
          'Check_Stock':trans_log['Check_Stock'],'Stock_Keeper':trans_log['Stock_Keeper'],
          'PO_Create':trans_log['PO_Create'],'Admin2':trans_log['Admin2'],
          'APAS repair':trans_log['APAS repair'],'Admin3':trans_log['Admin3'],
          'APAS approved':trans_log['APAS approved'],'Admin4':trans_log['APAS approved']
    },ignore_index=True)
    trans_log=[]
print('Done')

Done


In [20]:
rma_dict['FMSV2023060018']

'export (20).csv'

In [36]:
standard_format

{'RMA': 'FMSV2023090066',
 'SN': '1G402K223',
 'Receive': '2023-09-28',
 'Receiver': 'FFAP0104',
 'Inspection': '2023-09-29',
 'Inspector': 'FFAP0070',
 'Report': '2023-09-29',
 'Reporter': 'FFAP0070',
 'Verify': '2023-10-09',
 'Verifier': 'thongle',
 'Quotation': '2023-10-02',
 'Admin': 'FFAP0104',
 'Confirmation': '2023-10-05',
 'FSS': 'thongle',
 'Check_Stock': '2023-06-06',
 'Stock_Keeper': 'FFAP0066',
 'PO_Create': '2023-06-06',
 'Admin2': 'FFAP0066',
 'APAS repair': '',
 'Admin3': '',
 'APAS approved': '',
 'Admin4': ''}

In [45]:
# rma_check
rma_check ='FMSV2023020022'
file_csv = rma_dict[rma_check]
df = pd.read_csv(f'translog/{file_csv}',encoding = 'utf-8')
display(df)
read_translog(df)

,S/N,RMA#,Time,User,Station,Screen,Type,Updated Item(After),Updated Item(Before)
0,2C730K032,FMSV2023020022,2023/08/14 12:49:53,thongle,Inspection,Basic,Save,"Create:""2023/02/08 00:00:00"", Status Info:""PO ...","Create:""2023/02/08 16:53:29"", Status Info:""Par..."
1,2C730K032,FMSV2023020022,2023/06/06 11:25:51,FFAP0066,Inspection,Basic,Save,"Status Info:""Parts on Order"", Internal Note:""F...","Status Info:""Awating Internal Approval"", Inter..."
2,2C730K032,FMSV2023020022,2023/05/29 11:43:30,thongle,Inspection,Basic,Save,"Status Info:""Awating Internal Approval""","Status Info:"""""
3,2C730K032,FMSV2023020022,2023/02/14 12:20:09,FFAP0070,Inspection,Parts Selection,Save,"[11][Appr.:1, Parts No.:""350N102482C"", Delete:...",NaN
4,2C730K032,FMSV2023020022,2023/02/14 12:09:53,FFAP0070,Receive,Inspection,Completion,"Status:""Inspection"", Inspection:""2023/02/14 12...","Status:""Receive"", Inspection:"""", Inspector:"""",..."
5,2C730K032,FMSV2023020022,2023/02/14 12:08:58,FFAP0070,Receive,Parts Selection,Save,"[0][Appr.:1, Parts No.:""7A1333928F"", Delete:0,...",NaN
6,2C730K032,FMSV2023020022,2023/02/14 12:02:24,FFAP0070,Receive,Inspection,Save,"Repair Type:""Major"", FDA Question1:""No"", FDA Q...","Repair Type:"""", FDA Question1:"""", FDA Question..."
7,2C730K032,FMSV2023020022,2023/02/08 18:54:02,nguyenminh,Create,Shipping Incoming,Completion,"Status:""Receive"", Receive:""2023/02/08 18:54:02...","Status:""Create"", Receive:"""", RECIEVE_USER_NAME:"""""
8,2C730K032,FMSV2023020022,2023/02/08 18:53:57,nguyenminh,NaN,Basic,Create,"State:"""", City:"""", Customer:""FMSV00080"", Curre...",NaN


{'RMA': 'FMSV2023020022',
 'SN': '2C730K032',
 'Receive': '2023-02-08',
 'Receiver': 'nguyenminh',
 'Inspection': '2023-02-14',
 'Inspector': 'FFAP0070',
 'Report': '',
 'Reporter': '',
 'Verify': '',
 'Verifier': '',
 'Quotation': '',
 'Admin': '',
 'Confirmation': '',
 'FSS': '',
 'Check_Stock': '2023-06-06',
 'Stock_Keeper': 'FFAP0066',
 'PO_Create': '2023-06-06',
 'Admin2': 'FFAP0066',
 'APAS repair': '',
 'Admin3': '',
 'APAS approved': '',
 'Admin4': ''}

In [8]:
path = pathlib.Path().absolute()
today = dt.now().strftime('%y%m%d')
today

'231013'

In [43]:
file_name = os.path.join(path,'inspection',f'translog_{today}.xlsx')
full_translog.to_excel(file_name,index=False)

In [44]:

wb = xw.Book(file_name)
sh = wb.sheets('Sheet1')
sh.autofit()

In [12]:
full_translog[full_translog]

,RMA,SN,Receive,Receiver,Inspection,Inspector,Report,Reporter,Verify,Verifier,...,Confirmation,FSS,Check_Stock,Stock_Keeper,PO_Create,Admin2,APAS repair,Admin3,APAS approved,Admin4
0,FMSV2023020022,2C730K032,2023-02-08,nguyenminh,2023-02-14,FFAP0070,,,,,...,,,2023-06-06,FFAP0066,2023-06-06,FFAP0066,,,,
1,FMSV2023030069,1C741K218,2023-03-28,thongle,2023-03-28,thongle,,,,,...,,,2023-06-06,FFAP0066,2023-06-06,FFAP0066,,,,
2,FMSV2023090013,6C692K137,2023-09-08,tranminh,2023-09-12,FFAP0062,2023-09-12,FFAP0062,2023-09-12,thang,...,,,2023-06-06,FFAP0066,2023-06-06,FFAP0066,,,,
3,FMSV2023030075,1C603K041,2023-03-30,FFAP0064,2023-04-04,FFAP0070,2023-09-12,FFAP0062,2023-09-12,thang,...,,,2023-06-06,FFAP0066,2023-06-06,FFAP0066,,,,
4,FMSV2023030079,2V567K740,2023-03-30,tranminh,2023-04-04,FFAP0062,2023-09-12,FFAP0062,2023-09-12,thang,...,,,2023-06-06,FFAP0066,2023-06-06,FFAP0066,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,FMSV2023100020,1C692K439,2023-10-11,FFAP0070,2023-10-11,FFAP0070,2023-10-09,Nguyen,2023-10-09,thongle,...,2023-10-05,thongle,2023-10-09,thongle,2023-10-09,thongle,,,,
362,FMSV2023100021,1G348K862,2023-10-11,FFAP0070,2023-10-11,FFAP0070,2023-10-09,Nguyen,2023-10-09,thongle,...,2023-10-05,thongle,2023-10-09,thongle,2023-10-09,thongle,,,,
363,FMSV2023100022,1C642K481,2023-10-11,FFAP0070,2023-10-11,FFAP0070,2023-10-09,Nguyen,2023-10-09,thongle,...,2023-10-05,thongle,2023-10-09,thongle,2023-10-09,thongle,,,,
364,FMSV2023030060,2V644K028,2023-03-23,nguyenminh,2023-03-31,FFAP0062,2023-10-09,Nguyen,2023-10-09,thongle,...,2023-10-05,thongle,2023-06-23,FFAP0066,2023-08-09,thang,,,,


In [ ]:
#
sn = df['S/N'][0]
rma = df['RMA#'][0]

In [ ]:
print(rma)